In [ ]:
# Blog 1 Code for RDD plot, RDD visualization
# Along with all other statistics used in paper

################################################################################
################################################################################
# Packages
################################################################################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.stats.mstats import winsorize



################################################################################
# Data
################################################################################
df1 = pd.read_csv('ManuCounty1721.csv')

df21 = pd.read_csv('/content/conreport2021.csv')
df22 = pd.read_csv('/content/conreport2020.csv')
df23 = pd.read_csv('/content/conreport2019.csv')
df24 = pd.read_csv('/content/conreport2018.csv')
df25 = pd.read_csv('/content/conreport2017.csv')

################################################################################
# Data Cleaning
################################################################################
# Adding year to each df
df21['Year (YEAR)'] = 2021
df22['Year (YEAR)'] = 2020
df23['Year (YEAR)'] = 2019
df24['Year (YEAR)'] = 2018
df25['Year (YEAR)'] = 2017


df2 = pd.concat([df21, df22, df23, df24, df25]).reset_index()


df2 = df2.rename(columns = {'Year (YEAR)': 'Year'})
df2['Year'] = pd.to_numeric(df2['Year'])

# Abbreviation dictionary
state_abbrev_to_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}




# String maninpulation to make the county and state columns consistent
df2['StateABV'] = df2['County'].str.extract(r',\s([A-Z]{2})')
df2['State'] = df2['StateABV'].map(state_abbrev_to_name)
df2['County'] = df2['County'].str.extract(r'^(.+?),')
df2['County'] = df2['County'].str.replace(r'County', '', regex=True).str.strip()

# merge into main df
df = df1.merge(df2, on = ['State', 'County', 'Year'], how = 'inner')
df.replace('.', np.nan, inplace=True)

# Renaming
df = df.rename(columns = {'NES Receipts (in thousands)': 'Receipts',
                          'PM2.5 Weighted Mean 24-hr': 'PM25',
                          'CBP Annual Payroll (in thousands)': 'Payroll',
                          'CBP Employment, including March 12th': 'Employees',
                          'Total CBP and NES Establishments': 'Firms',
                          '2017 NAICS Code': 'NAICS',
                          'Ozone 4th Max 8-hr': 'Ozone'})


# Transforming variables of interest
# These where coded as strings with commas within numbers > 999
df['Receipts'] = df['Receipts'].str.replace(',', '', regex=False)
df['Receipts'] = pd.to_numeric(df['Receipts'], errors='coerce')

df['Employees'] = df['Employees'].str.replace(',', '', regex=False)
df['Employees'] = pd.to_numeric(df['Employees'], errors='coerce')

df['Payroll'] = df['Payroll'].str.replace(',', '', regex=False)
df['Payroll'] = pd.to_numeric(df['Payroll'], errors='coerce')

df['Firms'] = df['Firms'].str.replace(',', '', regex=False)
df['Firms'] = pd.to_numeric(df['Firms'], errors='coerce')


df['PM25'] = pd.to_numeric(df['PM25'], errors='coerce')
df['Ozone'] = pd.to_numeric(df['Ozone'], errors='coerce')

df = df.dropna(subset = ['Receipts', 'PM25', 'Firms', 'Ozone']).reset_index()

################################################################################
# Outcome Variables
################################################################################
df['PM25_sqr'] = df['PM25']**2
df['Ozone_sqr'] = df['Ozone']**2

df['logReceipts'] = np.log(df['Receipts'])
df['ReceiptsPerFirm'] = df['Receipts'] / df['Firms']
df['logReceiptsPerFirm'] = np.log(df['ReceiptsPerFirm'])

################################################################################
# PARAMETERS
################################################################################
cutoff = 0.071

runvar = 'Ozone'
outvar = 'Receipts'
################################################################################
# REMOVING OUTLIERS
################################################################################
u = np.quantile(df[outvar], 0.95)
l = np.quantile(df[outvar], 0.05)

df = df[df[outvar] < u]
df = df[df[outvar] > l]

u = np.quantile(df[runvar], 0.95)
l = np.quantile(df[runvar], 0.05)

df = df[df[runvar] < u]
df = df[df[runvar] > l]

################################################################################
# NAICS Filtering
################################################################################
# Transportation Manufacuturing NAICS
target_naics = ['336']

df = df[df['NAICS'].isin(target_naics)]

################################################################################
# MODEL CREATION
################################################################################
# Defining Treatment
df['treatment'] = (df[runvar] >= cutoff).astype(int)

# Intial model without weighting
model = smf.wls('Receipts ~ Ozone + treatment:Ozone', data=df).fit()

# Weighting as done by Walker, 2017
residuals = model.resid
resid_sq = residuals**2


fitted_vals = model.fittedvalues
aux_reg = sm.OLS(resid_sq, sm.add_constant(fitted_vals)).fit()

estimated_var = aux_reg.fittedvalues

#Inverse
weights = 1 / estimated_var

# Final model with weighting
model = smf.wls('Receipts ~ Ozone + treatment + Ozone:treatment', data=df, weights = weights).fit()
print(model.summary())



################################################################################
# RDD VISUALIZATION
################################################################################

# Getting red and blue colors for visualization
color_below = '#2E86AB'
color_above = '#A23B72'
cutoff_color = '#F18F01'

################################################################################
# PLOT 1: Main RDD Plot
################################################################################
# Make subplot canvas
fig1, ax1 = plt.subplots(1, 1, figsize=(12, 8))

# Scatter plot of raw data
ax1.scatter(df[df[runvar] < cutoff][runvar], df[df[runvar] < cutoff][outvar],
           alpha=0.6, s=30, color=color_below, label='Below Cutoff')
ax1.scatter(df[df[runvar] >= cutoff][runvar], df[df[runvar] >= cutoff][outvar],
           alpha=0.6, s=30, color=color_above, label='Above Cutoff')

# Add fitted regression lines
ozone_below = df[df[runvar] < cutoff][runvar]
ozone_above = df[df[runvar] >= cutoff][runvar]

# Defining best fit lines for treatment and control
if len(ozone_below) > 0:
    z_below = np.polyfit(ozone_below, df[df[runvar] < cutoff][outvar], 2)
    p_below = np.poly1d(z_below)
    x_below = np.linspace(ozone_below.min(), cutoff, 100)
    ax1.plot(x_below, p_below(x_below), color=color_below, linewidth=3, alpha=0.8)

if len(ozone_above) > 0:
    z_above = np.polyfit(ozone_above, df[df[runvar] >= cutoff][outvar], 2)
    p_above = np.poly1d(z_above)
    x_above = np.linspace(cutoff, ozone_above.max(), 100)
    ax1.plot(x_above, p_above(x_above), color=color_above, linewidth=3, alpha=0.8)

# Creating cutoff line
ax1.axvline(x=cutoff, color=cutoff_color, linestyle='--', linewidth=3,
           label=f'Cutoff ({cutoff:.3f} ppm)')

# Labeling
ax1.set_xlabel('Ozone 4th Max 8-hr (ppm)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Manufacturing Receipts (thousands $)', fontsize=14, fontweight='bold')
ax1.set_title('Regression Discontinuity: Ozone Effects on Manufacturing Receipts\n(NAICS 336 - Transportation Equipment)',
              fontsize=16, fontweight='bold', pad=20)
ax1.legend(fontsize=12, loc='upper right')
ax1.grid(True, alpha=0.3)

# Format y-axis to show values in millions (not thousands like the data)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.1f}M'))

# Creating model and graphic variables
stats_text = f"""Treatment Effect: ${model.params['treatment']:.0f}k
Standard Error: ${model.bse['treatment']:.0f}k
P-value: {model.pvalues['treatment']:.3f}
R²: {model.rsquared:.3f}
N = {len(df):,}"""

# Adding stats, above, to visual
ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, fontsize=11,
         verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5",
         facecolor="white", alpha=0.9, edgecolor='gray'))

plt.tight_layout()
plt.show()

################################################################################
# PLOT 2: Histogram of Running Variable Around Cutoff
# Was not used in blog as of now
# Robust testing, for validation
# Not included in paper for length
################################################################################
fig2, ax2 = plt.subplots(1, 1, figsize=(12, 6))

# Create histogram
n_bins = 40
ax2.hist(df[df[runvar] < cutoff][runvar], bins=n_bins, alpha=0.7, color=color_below,
         label=f'Below Cutoff (n={sum(df[runvar] < cutoff):,})', density=False, edgecolor='white')
ax2.hist(df[df[runvar] >= cutoff][runvar], bins=n_bins, alpha=0.7, color=color_above,
         label=f'Above Cutoff (n={sum(df[runvar] >= cutoff):,})', density=False, edgecolor='white')

# Add cutoff line
ax2.axvline(x=cutoff, color=cutoff_color, linestyle='--', linewidth=3,
           label=f'Cutoff ({cutoff:.3f} ppm)')

# Formatting
ax2.set_xlabel('Ozone 4th Max 8-hr (ppm)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=14, fontweight='bold')
ax2.set_title('Distribution of Running Variable Around Cutoff\n(Testing for Manipulation)',
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

# Add interpretation text
interpretation_text = """No evidence of manipulation around cutoff
Smooth distribution suggests valid RDD design"""

ax2.text(0.98, 0.98, interpretation_text, transform=ax2.transAxes, fontsize=11,
         verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.9, edgecolor='gray'))

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'ManuCounty1721.csv'

In [ ]:
# Some groupby statistics used in blog
dfg = df.groupby('NAICS')['Receipts'].agg('sum').reset_index()
dfg = dfg.sort_values(by = 'Receipts', ascending = False)
dfg

temp = temp[temp['NAICS'].str.contains("^[0-9]{3}$")]
temp = temp.groupby('NAICS').agg({
    'Receipts': 'sum',
    'Ozone': 'mean'
}).reset_index()


temp = temp.sort_values(by = ['Receipts'], ascending = False)
not_sig = ['322', '324', '316', '314', '333', '323', '339', '311', '332']
temp = temp[~temp['NAICS'].isin(not_sig)]
temp

,NAICS,Receipts,Ozone
2,315,2854515.0,0.068409
11,337,2795436.0,0.066536
3,321,2656015.0,0.066103
10,336,2642238.0,0.067463
4,325,2407911.0,0.066814
8,334,1699101.0,0.067708
9,335,1414659.0,0.068579
0,312,1381995.0,0.067506
6,327,1241369.0,0.066834
5,326,965074.0,0.068358


Background Statistics On Only Trans

In [ ]:
# Summary stats I used for blog
df_trans = df[df['NAICS'] == '336'].copy()


cols_to_convert = ['Firms', 'Employees', 'Receipts', 'Payroll',
                'Ozone']
for col in cols_to_convert:
    df_trans[col] = pd.to_numeric(df_trans[col], errors='coerce')


summary_cols = ['Firms', 'Employees', 'Receipts', 'Payroll',
                'Ozone']

summary_table = df_trans[summary_cols].describe().T

summary_table = summary_table.round(2)


print(summary_table[['mean', 'std', 'min', '25%', '50%', '75%', 'max']])


                mean        std    min       25%       50%        75%  \
Firms          56.22      93.12   6.00     17.00     27.00      53.00   
Employees    3189.99    6576.79   4.00    243.75    917.50    2749.50   
Receipts     2272.82    4524.69  10.00    417.00    961.00    2120.50   
Payroll    250591.71  591477.18  45.00  13999.75  52637.50  182430.00   
Ozone           0.07       0.01   0.04      0.06      0.07       0.07   

                  max  
Firms          882.00  
Employees    49853.00  
Receipts     42434.00  
Payroll    7086075.00  
Ozone            0.12  


PearsonRResult(statistic=np.float64(0.7773619050859402), pvalue=np.float64(0.0))